In [1]:
# https://www.lab2050.org/

In [24]:
import json
import os 
import requests
import time
from datetime import datetime,timedelta
from datetime import date
import re
import sys
import urllib, urllib.request, urllib.parse
import random
from scrawl import *
    
# Date and time
start_time = time.time()
current_time = datetime.now().strftime("%H-%M-%S")
created_on = date.today().strftime("%Y-%m-%d")

# client_id = sys.argv[1]
client_id = '5f69d22ef472d6646f577fa6'  # Europe
site = 'lab2050_org_publication'
c = Crawl()  # creating object

# create directories to store logs.
log_path = c.create_directories(project_path, client_id, site)

## creating pdf directories
pdf_directory = c.create_pdf_directories(project_path, site)
# create image directories
image_directory = c.create_image_directories(project_path)

# logger
logger = log_func(log_path, created_on, current_time)
logger.info("Process Started ...\n")

# initialize variables
skipped_due_to_headline = 0
skipped_due_to_content = 0
skipped_due_to_date = 0
missing_overall_tonality = 0
no_of_data = 0
duplicate_data = 0  
unable_to_fetch_article_url = 0
unable_to_fetch_cat_url = 0
unable_to_download_pdf = 0
publish_source = 'lab2050.org'
country = 'Korea'
language = 'English'
images_path = []

home_page = c.download_page('https://medium.com/lab2050')
# home_page = c.scrap('pane-block-47\s*pane-block"\s*>(.*?)<div\s*class="col-md-2">',home_page)
for _ in home_page.split('<li class="collectionHeader-navItem ')[1:2]: 
    
    cat_url = c.scrap('href="(.*?)"', _)  
    print(cat_url)
#     continue

    logger.info(f'Fetching cat url  {cat_url}\n')
    time.sleep(5)
    cat_page = c.download_page(cat_url) 
#     print(cat_page)
#     continue
    
    if cat_page.startswith('Unable to fetch'):
        logger.info(cat_page) 
        unable_to_fetch_cat_url += 1
        continue    
    for i in cat_page.split('<div class="u-lineHeightBase postItem')[1:]:
        # source_link
#         print(i)
#         break
#         link = c.scrap('<span\s*class="u-textScreenReader">(.*?)</span>',i)  # it wil take the name of the url
#         link = re.sub("\s+|;|'|”","-",link)  # subsituting multiple thing together
#         link = re.sub('\“','',link)
#         source_link = c.scrap('<a\s*href="(.*?)"', i)   # right   it wil url , inthat url i wwill make chnages
#         source_link =  re.sub("\?(.*)",'',source_link)  # right
        
#         source_link = re.sub('lab2050(.*)-',f'lab2050/{link}',source_link)
#         print(source_link)
     

        # handle duplicates
        source_link_query = {'source_link':source_link}
        dic = cl_data.find_one(source_link_query,{'source_link': 1}) 
        if dic:
            duplicate_data += 1
            continue

        time.sleep(random.randint(1,3))

        logger.info(f'Fetching {source_link}\n')
        time.sleep(7)
        page = c.download_page(source_link)   # here the page wil get download
        time.sleep(4)
        if page.startswith('Unable to fetch'):     
            logger.info(page) # writes error message with error code
            unable_to_fetch_article_url += 1
            continue    

        source_headline = c.scrap('<span\s*class="u-textScreenReader">(.*?)</span>',i)
        source_headline = re.sub('&.*?;','',source_headline)
        print(source_headline)
        # skip if headline not found
        if not source_headline:
            logger.info(f'Skipping due to headline {source_link}\n')
            skipped_due_to_headline += 1
            continue
            
         # Date and time
        #"datePublished":"2021-04-26T05:10:03.920Z"
        pub_date, publish_time = '', ''
        try:
            date_time_str = c.scrap('datePublished":"(.*?)\.', page) 
            date_time_str = re.sub('[^\w+]', '', date_time_str)  
            date_time_obj = datetime.strptime(date_time_str, '%Y%m%dT%H%M%S')
            ist_date_time = date_time_obj - timedelta(hours = 3,minutes = 30)  
            ist_date_time = ist_date_time.strftime('%Y-%m-%d %H:%M:%S')
            pub_date = ist_date_time[:10]
            publish_time = ist_date_time[11:]
            print(pub_date)
        except:
            pass
        
        # skip null date
        if not pub_date:
            logger.info(f'Skipping due to date {source_link}\n')            
            skipped_due_to_date += 1
            continue

        # break if date is not today's date
#         if pub_date != created_on:
#             break    


        # source_content         
        source_content= c.scrap('class="hs">(.*?)</article>',page)   
#         if not source_content:
#             source_content= c.scrap('div\s*class="post-data-container">(.*?)<div\s*class="m-b-',page)
#         if not source_content:
#             source_content= c.scrap('div\s*class="post-data-container">(.*?)<div\s*class="col-md-4">',page)
        source_content = re.sub('&.*?;','',source_content)
        source_content = c.strip_html(source_content) 
        print(source_content)
        continue
        if not source_content:
            logger.info(f'Skipping due to content {source_link}\n')            
            skipped_due_to_content += 1
            continue

        
        journalist =c.scrap("'author-name':'(.*?)'",page)

        if not journalist: journalist = 'NA'

        # current date and time 00
        harvest_time = datetime.now().strftime("%H:%M:%S")

        # temp link
        temp_link = source_link

        # headline and content 
        headline = source_headline
        content = source_content

        # overall_tonality
        overall_tonality = ''

        # word count
        word_count = len((source_headline + ' ' + source_content).split())

        html_content = ''

        # image_urls
        image_urls = []
    
        
        # pdf url
        for i in page.split('href=')[1:]:
            pdf_url = c.scrap('"(.*?)"',i)
            if pdf_url.endswith('.pdf'):
                pdf_name = c.scrap('.*\/(.*)',pdf_url)
                if ('Shifrinson' in pdf_name) or ('australiaapril' in pdf_name):
                    continue
                pdf_path = f'{pdf_directory}/{pdf_name}'        
               # download pdf
                pdf = c.download_pdf(pdf_url, pdf_path)
                if pdf.startswith('Unable to fetch'):
                    logger.info(pdf) # writes error message with error code
                    unable_to_download_pdf += 1
                    continue

        # storing the above data in a dictionary
        clientdata ={
                        "client_master" : client_id, 
                        "articleid":client_id,
                        "medium":'Web' ,
                        "searchkeyword":[],
                        "entityname" : [] ,
                        "process_flage":"1",
                        "na_flage":"0",
                        "na_reason":"",
                        "qc_by":"",
                        "qc_on":"",
                        "location":"",
                        "spokeperson":"",
                        "quota":"",
                        "overall_topics":"",
                        "person":"",
                        "overall_entites":"",
                        "overall_tonality": overall_tonality,
                        "overall_wordcount":word_count,
                        "article_subjectivity":"",
                        "article_summary":"",
                        "pub_date":pub_date,
                        "publish_time":publish_time,
                        "harvest_time":harvest_time,
                        "temp_link":temp_link,
                        "publish_source": publish_source,
                        "programme":'null',
                        "feed_class":"News",
                        "publishing_platform":"",
                        "klout_score":"",
                        "journalist":journalist,
                        "headline":headline,
                        "content":content,
                        "source_headline":source_headline,
                        "source_content":source_content,
                        "language":language,
                        "presence":'null',
                        "clip_type":'null',
                        "prog_slot":'null',
                        "op_ed":'0',
                        "location_mention":'',
                        "source_link":source_link,
                        "author_contact":'',
                        "author_emailid":'',
                        "author_url":'',
                        "city":'',
                        "state":'',
                        "country":country,
                        "source":publish_source,
                        "foot_fall":'',
                        "created_on":created_on,
                        "active":'1',
                        'crawl_flag':2,
                        "images_path":images_path,
                        "html_content":html_content,
                        "pdf_url": pdf_url,
                        "pdf_name": pdf_name,
                        "pdf_path":pdf_path
                    }
#         cl_data.insert_one(clientdata)  
        no_of_data += 1


logger.info('Iteration complete\n')   
logger.info(f'Number of data: {no_of_data}\n')
logger.info(f'Duplicate data: {duplicate_data}\n')
logger.info(f'Unable to fetch cat url: {unable_to_fetch_cat_url}\n')
logger.info(f'Unable to fetch article url: {unable_to_fetch_article_url}\n')
logger.info(f'Skipped due to headline: {skipped_due_to_headline}\n')
logger.info(f'Skipped due to content: {skipped_due_to_content}\n')
logger.info(f'Skipped due to date: {skipped_due_to_date}\n')
logger.info(f'Unable to download pdf: {unable_to_download_pdf}\n')
logger.info(f'country: {country}\n')
logger.info(f'language: {language}\n')
logger.info(f'Processing finished in {time.time() - start_time} seconds.\n')

https://medium.com/lab2050/story/home
https://medium.com/lab2050/기본소득-다음은-경제의-전환-7a3c1885d2c2
“기본소득 다음은 경제의 전환”
2021-04-26
2021ë…„ 2ì›” ì •ê¸°ì�´ì‚¬íšŒë¥¼ í†µí•´ ì •ê±´í™” í•œì‹ ëŒ€ ê²½ì œí•™ê³¼ êµ�ìˆ˜ê°€ LAB2050ì�˜ ìƒˆë¡œìš´ ì�´ì‚¬ì�¥ìœ¼ë¡œ ì·¨ì�„í–ˆë‹¤. ë…¸ë�™ê²½ì œí•™ì��ì�¸ ì •ê±´í™” ì�´ì‚¬ì�¥ì�€ ì‚¬íšŒì �ê²½ì œ, ì‚¬íšŒí˜�ì‹ , ì§€ì—­ì‚¬íšŒì™€ ëŒ€í•™ ê°„ í˜‘ë ¥ ë“±ì—�ë�„ ê´€ì‹¬ì�„ ê°–ê³  ì—°êµ¬, ì‹¤ì²œí•´ ì™”ë‹¤. ê²½ì œì�˜ ìƒ�íƒœì � ì „í™˜, ë¯¼ê°„ ì‹±í�¬íƒ±í�¬ì�˜ ì§€ì†�ê°€ëŠ¥ì„±, ì‚¬íšŒí˜�ì‹ ì�„ ìœ„í•´ í•„ìš”í•œ ê³¼ì œ ë“± LAB2050ì�´ ì•�ìœ¼ë¡œ ì¶”êµ¬í•  ë°©í–¥ì—� ëŒ€í•´ ì • ì�´ì‚¬ì�¥ê³¼ ì�´ì•¼ê¸°ë¥¼ ë‚˜ëˆ´ë‹¤. ì�¸í„°ë·°ëŠ” 2021ë…„ 4ì›” 13ì�¼(í™”), ì˜¤í›„ 3ì‹œë¶€í„° ì•½ 2ì‹œê°„ ë�™ì•ˆ LAB2050 íšŒì�˜ì‹¤ì—�ì„œ ê¹€ë¯¼ì§„ ì—°êµ¬ì›�ì�´ ì§„í–‰í–ˆë‹¤.â€œì—°êµ¬ì™€ ëŒ€ì¤‘êµ�ìœ¡ì�´ í•¨ê»˜ ì�´ì–´ì§„ ì � ì�¸ìƒ�ì �, ì•�ìœ¼ë¡œ ìƒˆë¡œìš´ ì‹¤í—˜ì�„ í†µí•´ ì‚¬íšŒì�˜ ì£¼ë¥˜ë¥¼ ë°”ê¾¸ëŠ” ì „í™˜ì�´ ê¸°ëŒ€ ë�¼â€�(ë¬¸) ì�´ì‚¬ì�¥ë‹˜ê»˜ì„œëŠ” ê·¸ë�™ì•ˆ ì–´ë–¤ ë¶„ì•¼ì—� ê´€ì‹¬ì�„ ë‘�ì…¨ë‚˜ìš”? ê´€ì‹¬ ë¶„ì•¼ì™

https://medium.com/lab2050/궁금한-당신이-물으면-답해-드립니다-:-클럽하우스-속-기본소득-이야기2a8a127d2044
https://medium.com/lab2050/클럽하우스에서-정책토론을-할-수-있을까?4bbf10f7905b
